# Graph Convolutional Network by Kipf and Welling

## Imports

In [9]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

In [10]:
class LinearModule(nn.Module):
    """The linear transformation part of the GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(LinearModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation # This is the activation function

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

In [11]:
class GCN(nn.Module):
    """A GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = LinearModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(message_func=fn.copy_src(src='h', out='m'), reduce_func=fn.sum(msg='m', out='h'))
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [62]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gcn1 = GCN(1433, 16, F.relu)
        self.gcn2 = GCN(16, 7, lambda x: F.log_softmax(x,1))
        self.dropout = nn.Dropout(0.2)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        x = self.dropout(x)
        x = self.gcn2(g, x)
        return x

In [15]:
from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    mask = th.ByteTensor(data.train_mask)
    g = data.graph

    # add self loop
    g.remove_edges_from(nx.selfloop_edges(g))
    g = DGLGraph(g)
    g.add_edges(g.nodes(), g.nodes())
    
    return g, features, labels, mask

In [21]:
#g, features, labels, mask = load_cora_data()
1-mask

tensor([0, 0, 0,  ..., 1, 1, 1], dtype=torch.uint8)

In [63]:
import time
import numpy as np

net = Net()
#print(net)

optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
net.train() # Set to training mode (use dropout)

dur = []
for epoch in range(90):
    if epoch >=3:
        t0 = time.time()

    net.eval()
    prediction = net(g, features)
    val_loss = F.nll_loss(prediction.detach()[1-mask], labels[1-mask])
    net.train()
    #val_loss = F.nll_loss(logits.detach()[1-mask], labels[1-mask])

    logits = net(g, features)
    loss = F.nll_loss(logits[mask], labels[mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Val.Loss {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), val_loss.item(), np.mean(dur)))

Epoch 00000 | Loss 1.9975 | Val.Loss 1.9568 | Time(s) nan
Epoch 00001 | Loss 1.9753 | Val.Loss 1.9422 | Time(s) nan
Epoch 00002 | Loss 1.9476 | Val.Loss 1.9287 | Time(s) nan
Epoch 00003 | Loss 1.9187 | Val.Loss 1.9169 | Time(s) 0.1580
Epoch 00004 | Loss 1.9089 | Val.Loss 1.9053 | Time(s) 0.2475
Epoch 00005 | Loss 1.8851 | Val.Loss 1.8934 | Time(s) 0.2233
Epoch 00006 | Loss 1.8574 | Val.Loss 1.8813 | Time(s) 0.2180
Epoch 00007 | Loss 1.8291 | Val.Loss 1.8692 | Time(s) 0.2178
Epoch 00008 | Loss 1.7967 | Val.Loss 1.8577 | Time(s) 0.2153
Epoch 00009 | Loss 1.7946 | Val.Loss 1.8455 | Time(s) 0.2093
Epoch 00010 | Loss 1.7597 | Val.Loss 1.8330 | Time(s) 0.2285
Epoch 00011 | Loss 1.7313 | Val.Loss 1.8202 | Time(s) 0.2270
Epoch 00012 | Loss 1.7301 | Val.Loss 1.8065 | Time(s) 0.2229
Epoch 00013 | Loss 1.6733 | Val.Loss 1.7921 | Time(s) 0.2355
Epoch 00014 | Loss 1.6348 | Val.Loss 1.7772 | Time(s) 0.2303
Epoch 00015 | Loss 1.6543 | Val.Loss 1.7623 | Time(s) 0.2289
Epoch 00016 | Loss 1.6279 | Val.L

## Evaluation

In [85]:
from sklearn.metrics import accuracy_score as acc
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()

pred_sets = {"All ":final_prediction,"Train":final_prediction[mask],"Test":final_prediction[1-mask]}
label_sets = {"All ":labels,"Train":labels[mask],"Test":labels[1-mask]}
eval_functions = {"NLL-Loss":lambda y,x: F.nll_loss(x,y),"Accuracy":lambda y,x: acc(y,x.numpy().argmax(axis=1))}

for name,func in eval_functions.items():
    eval_message = f"\n{name}:\n"
    for subset in pred_sets.keys():
        eval_message += f" {subset}: {func(label_sets[subset],pred_sets[subset]):.4f} |"
    print(eval_message)


NLL-Loss:
 All : 1.1305 | Train: 0.6747 | Test: 1.1553 |

Accuracy:
 All : 0.7740 | Train: 0.9571 | Test: 0.7640 |
